In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import mean_squared_error, r2_score, confusion_matrix, accuracy_score, roc_auc_score, roc_curve, classification_report
import matplotlib.pyplot as plt
from sklearn.preprocessing import scale, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [0]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

In [3]:
from google.colab import files

uploaded = files.upload()

Saving diabetes.csv to diabetes.csv


In [0]:
df=pd.read_csv("./diabetes.csv")

In [5]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [0]:
y=df["Outcome"]
X=df.drop(["Outcome"], axis=1)
X_train, X_test, y_train, y_test=train_test_split(X,
                                                  y,
                                                  test_size=0.30,
                                                  random_state=42)

**Model&Tahmin**

In [0]:
mlpc_model=MLPClassifier().fit(X_train, y_train)

In [8]:
mlpc_model.coefs_

[array([[-2.10171717e-01,  5.71505304e-03, -5.36570114e-02,
         -1.39238503e-01, -2.36962001e-01,  7.73248664e-02,
          2.14539021e-03,  5.91948155e-02,  2.35636866e-01,
         -2.61750905e-01,  1.37138818e-02, -8.36667509e-02,
          3.08971604e-02,  2.87538277e-02,  3.37252810e-02,
         -5.59796784e-03,  6.55154664e-02,  1.11922837e-01,
         -1.94506466e-01,  1.74461213e-03, -2.12527504e-01,
          8.71779645e-02,  1.04906668e-02, -1.82273423e-01,
         -2.02063273e-02,  1.12464574e-01, -1.45350699e-01,
          9.92909978e-03, -2.15481401e-01, -4.37571046e-02,
          2.42053931e-01, -2.94094808e-02, -4.83971590e-03,
         -2.55073634e-01,  1.52209416e-01, -9.48196745e-02,
          1.60007256e-01,  5.16788620e-02, -1.55405350e-01,
         -6.81766576e-03,  1.86261865e-01, -6.35930580e-02,
         -1.84966721e-01,  1.16547038e-02, -1.74401954e-01,
         -1.92182508e-01,  2.47529363e-02, -6.66085078e-02,
         -3.10397302e-02, -4.07909580e-0

In [0]:
y_pred=mlpc_model.predict(X_test)

In [11]:
accuracy_score(y_test, y_pred)

0.7229437229437229

**Model tuning**

In [0]:
mlpc=MLPClassifier(solver="lbfgs")

In [0]:
mlpc_params={"alpha":[1,5,0.1,0.01,0.03,0.005,0.0001],
             "hidden_layer_sizes":[(10,10), (100,100,100),(100,100),(3,5)]}

In [16]:
mlpc_cv_model=GridSearchCV(mlpc,mlpc_params, cv=10, n_jobs=-1, verbose=2).fit(X_train, y_train)

Fitting 10 folds for each of 28 candidates, totalling 280 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:    6.5s
[Parallel(n_jobs=-1)]: Done 280 out of 280 | elapsed:   34.2s finished


In [18]:
mlpc_cv_model

GridSearchCV(cv=10, error_score=nan,
             estimator=MLPClassifier(activation='relu', alpha=0.0001,
                                     batch_size='auto', beta_1=0.9,
                                     beta_2=0.999, early_stopping=False,
                                     epsilon=1e-08, hidden_layer_sizes=(100,),
                                     learning_rate='constant',
                                     learning_rate_init=0.001, max_fun=15000,
                                     max_iter=200, momentum=0.9,
                                     n_iter_no_change=10,
                                     nesterovs_momentum=True, power_t=0.5,
                                     random_state=None, shuffle=True,
                                     solver='lbfgs', tol=0.0001,
                                     validation_fraction=0.1, verbose=False,
                                     warm_start=False),
             iid='deprecated', n_jobs=-1,
             param_grid=

In [17]:
mlpc_cv_model.best_params_

{'alpha': 1, 'hidden_layer_sizes': (10, 10)}

In [0]:
#final model
mlpc_tuned=MLPClassifier(solver="lbfgs", alpha=0.0001, hidden_layer_sizes=(3,5)).fit(X_train, y_train)

In [0]:
y_pred=mlpc_tuned.predict(X_test)

In [21]:
accuracy_score(y_test, y_pred)

0.6536796536796536

Veri setleri genellikle homojendir. Ama ağaca dayalı algoritmalar heterojen veri setleriyle daha iyi çalışır ayrıştırıcılığı yakalamaya çalıştığından dolayı. Burada da standartlaşma işlemi yapılırsa daha iyi olacaktır. 

In [0]:
#standartlaştırma için üste aşg kodlar eklenir.
#scaler=StandardScaler()
#scaler.fit(X_train)
#X_train=scaler.transform(X_train)
#scaler.fit(X_test)
#X_test=scaler.transform(X_test) 

#çalıştırıldığında score artar. Homojen veri setleriyle çalışmayı sevdiğinden dolayı standartlaştırma yapılmaldıır.